In [1]:
import random
import os


import pandas as pd
import numpy as np
import tensorflow as tf
import neuronperf as npf
import neuronperf.tensorflow
from tensorflow.keras.utils import to_categorical
from util import benchmark
import csv

SEED = 1

2024-03-19 02:25:31.943249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 02:25:32.059059: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/lib
2024-03-19 02:25:32.059078: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-19 02:25:32.086203: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 02:25:3

In [2]:
# load basic model 
model_dir = 'model/mmoe'   
model = tf.keras.models.load_model(model_dir)

2024-03-19 02:25:33.291173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def data_preparation():
    # The column names are from
    # https://www2.1010data.com/documentationcenter/prod/Tutorials/MachineLearningExamples/CensusIncomeDataSet.html
    column_names = ['age', 'class_worker', 'det_ind_code', 'det_occ_code', 'education', 'wage_per_hour', 'hs_college',
                    'marital_stat', 'major_ind_code', 'major_occ_code', 'race', 'hisp_origin', 'sex', 'union_member',
                    'unemp_reason', 'full_or_part_emp', 'capital_gains', 'capital_losses', 'stock_dividends',
                    'tax_filer_stat', 'region_prev_res', 'state_prev_res', 'det_hh_fam_stat', 'det_hh_summ',
                    'instance_weight', 'mig_chg_msa', 'mig_chg_reg', 'mig_move_reg', 'mig_same', 'mig_prev_sunbelt',
                    'num_emp', 'fam_under_18', 'country_father', 'country_mother', 'country_self', 'citizenship',
                    'own_or_self', 'vet_question', 'vet_benefits', 'weeks_worked', 'year', 'income_50k']

    # Load the dataset in Pandas
    train_df = pd.read_csv(
        'data/census-income.data.gz',
        delimiter=',',
        header=None,
        index_col=None,
        names=column_names
    )
    other_df = pd.read_csv(
        'data/census-income.test.gz',
        delimiter=',',
        header=None,
        index_col=None,
        names=column_names
    )

    # First group of tasks according to the paper
    label_columns = ['income_50k', 'marital_stat']

    # One-hot encoding categorical columns
    categorical_columns = ['class_worker', 'det_ind_code', 'det_occ_code', 'education', 'hs_college', 'major_ind_code',
                           'major_occ_code', 'race', 'hisp_origin', 'sex', 'union_member', 'unemp_reason',
                           'full_or_part_emp', 'tax_filer_stat', 'region_prev_res', 'state_prev_res', 'det_hh_fam_stat',
                           'det_hh_summ', 'mig_chg_msa', 'mig_chg_reg', 'mig_move_reg', 'mig_same', 'mig_prev_sunbelt',
                           'fam_under_18', 'country_father', 'country_mother', 'country_self', 'citizenship',
                           'vet_question']
    train_raw_labels = train_df[label_columns]
    other_raw_labels = other_df[label_columns]
    transformed_train = pd.get_dummies(train_df.drop(label_columns, axis=1), columns=categorical_columns)
    transformed_other = pd.get_dummies(other_df.drop(label_columns, axis=1), columns=categorical_columns)

    # Filling the missing column in the other set
    transformed_other['det_hh_fam_stat_ Grandchild <18 ever marr not in subfamily'] = 0

    # One-hot encoding categorical labels
    train_income = to_categorical((train_raw_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
    train_marital = to_categorical((train_raw_labels.marital_stat == ' Never married').astype(int), num_classes=2)
    other_income = to_categorical((other_raw_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
    other_marital = to_categorical((other_raw_labels.marital_stat == ' Never married').astype(int), num_classes=2)

    dict_outputs = {
        'income': train_income.shape[1],
        'marital': train_marital.shape[1]
    }
    dict_train_labels = {
        'income': train_income,
        'marital': train_marital
    }
    dict_other_labels = {
        'income': other_income,
        'marital': other_marital
    }
    output_info = [(dict_outputs[key], key) for key in sorted(dict_outputs.keys())]

    # Split the other dataset into 1:1 validation to test according to the paper
    validation_indices = transformed_other.sample(frac=0.5, replace=False, random_state=SEED).index
    test_indices = list(set(transformed_other.index) - set(validation_indices))
    validation_data = transformed_other.iloc[validation_indices]
    validation_label = [dict_other_labels[key][validation_indices] for key in sorted(dict_other_labels.keys())]
    test_data = transformed_other.iloc[test_indices]
    test_label = [dict_other_labels[key][test_indices] for key in sorted(dict_other_labels.keys())]
    train_data = transformed_train
    train_label = [dict_train_labels[key] for key in sorted(dict_train_labels.keys())]

    return train_data, train_label, validation_data, validation_label, test_data, test_label, output_info

train_data, train_label, validation_data, validation_label, test_data, test_label, output_info = data_preparation()

In [4]:
train_data = train_data.astype(float)
validation_data = validation_data.astype(float)
test_data = test_data.astype(float)

In [21]:
cpu_result = benchmark.benchmark_latency(model, test_data, 1, 100, compiled=False)
benchmark_dict = []
benchmark_dict.append({**cpu_result, "instance_type": "cpu"}) 
print(cpu_result)   

1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step
{'avg_ms': 41.199116940333624, 'std_ms': 1.6665755106864844, 'p95_ms': 45.87927430511627, 'p99_ms': 45.87927430511627}


In [22]:
# write results to csv
keys = benchmark_dict[0].keys()
with open(f'benchmmark_mmoe_cpu.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(benchmark_dict)

In [23]:
os.environ['NEURON_RT_NUM_CORES']='4'

In [24]:
neuron_dir = "./model/model_neuron"
neuron_model = tf.keras.models.load_model(neuron_dir)
neuron_model(test_data[:1])

[<tf.Tensor: shape=(49881, 2), dtype=float32, numpy=
 array([[0.995628  , 0.00437197],
        [0.9955972 , 0.00440288],
        [0.9955972 , 0.00440288],
        ...,
        [0.9955972 , 0.00440288],
        [0.9955972 , 0.00440288],
        [0.9955972 , 0.00440288]], dtype=float32)>,
 <tf.Tensor: shape=(49881, 2), dtype=float32, numpy=
 array([[0.9917604 , 0.00823959],
        [0.99749136, 0.00250862],
        [0.99749136, 0.00250862],
        ...,
        [0.99749136, 0.00250862],
        [0.99749136, 0.00250862],
        [0.99749136, 0.00250862]], dtype=float32)>]

In [25]:
# Benchmark and save results
neuron_result = benchmark.benchmark_latency(neuron_model, test_data, 1, 100, compiled=True)
benchmark_dict = []
benchmark_dict.append({**neuron_result, "instance_type": "neuron_core4"})    
print(neuron_result)

{'avg_ms': 225.32024730004196, 'std_ms': 0.6739220935662391, 'p95_ms': 226.22873630170943, 'p99_ms': 226.22873630170943}


In [20]:
# write results to csv
keys = benchmark_dict[0].keys()
with open(f'benchmmark_mmoe_neruon.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(benchmark_dict)